In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
import xcoll.geometry.c_init as box

import xcoll.geometry.c_init.bounding_box as bb

# %matplotlib ipympl
%matplotlib tk


# Frederik, you can go down to "Root tests".

# Bounding Box Tests

In [ ]:
sintC = np.sin(np.deg2rad(45))
cosTC = np.cos(np.deg2rad(45))
rC = 2
sin_tb = np.sin(0.)
cos_tb = np.cos(0)
projL = rC*(cos_tb*cosTC + sin_tb*sintC)
projW = rC*(cos_tb*sintC - sin_tb*cosTC) 
b1 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_tb, cos_tb=cos_tb, l=2,w=2, proj_l=projL, proj_w=projW)
#b1 = box.BoundingBox()
sintC = np.sin(np.deg2rad(11.309932))
cosTC = np.cos(np.deg2rad(11.309932))
rC = 5.099
sin_b = np.sin(np.deg2rad(45))
cos_b = np.cos(np.deg2rad(45))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b2 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=2.8284271247462, proj_l=projL, proj_w=projW)

sintC = np.sin(np.deg2rad(0.))
cosTC = np.cos(np.deg2rad(0.))
rC = 5.
sin_b = np.sin(np.deg2rad(45.))
cos_b = np.cos(np.deg2rad(45.))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b3 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=1.4142135623731, proj_l=projL, proj_w=projW)
#
#b1.set_params(box=b1, rC=rC, sin_tC=sintC, cos_tC=cosTC,l=2,w=2, sin_tb=sin_tb, cos_tb=cos_tb)
#b2.set_params(box=b2, rC=rC, sin_tC=sintC, cos_tC=cosTC,l=2.828427124746,w=2.8284271247462, sin_tb=sin_b, cos_tb=cos_b)
#b3.set_params(box=b3, rC=rC, sin_tC=sintC, cos_tC=cosTC,l=2.828427124746,w=1.4142135623731, sin_tb=sin_b, cos_tb=cos_b)

In [ ]:
sintC = np.sin(np.deg2rad(11.309932))
cosTC = np.cos(np.deg2rad(11.309932))
rC = 5.099
sin_b = np.sin(np.deg2rad(45))
cos_b = np.cos(np.deg2rad(45))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b2 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=2.8284271247462, proj_l=projL, proj_w=projW)

In [ ]:
sintC = np.sin(np.deg2rad(0.))
cosTC = np.cos(np.deg2rad(0.))
rC = 5.
sin_b = np.sin(np.deg2rad(45.))
cos_b = np.cos(np.deg2rad(45.))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b3 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=1.4142135623731, proj_l=projL, proj_w=projW)


In [ ]:
overlap1 = b1.overlaps(b2=b2)  # this should overlap
overlap2 = b1.overlaps(b2=b3)  # this should not overlap
overlap3 = b2.overlaps(b2=b3)  # this should overlap

print(f"Overlap1: {overlap1}")
print(f"Overlap2: {overlap2}")
print(f"Overlap3: {overlap3}")

In [ ]:
test = np.array([[13, 1], [11, 10], [4, 5], [6, 7], [8, 9]])

print(test[1,0])
print(test[0,0])
max_x_part = test[np.argmax(test[:, 0])]
max_y_part = test[np.argmax(test[:, 1])]

print("Part with biggest x value:", max_x_part)
print("Part with biggest y value:", max_y_part)

In [ ]:
def boundingBoxOverlap_test(b1,b2):
    """
    b1 is an array with coordinates of all vertices of the bounding box
    b2 is an array with coordinates of all vertices of the bounding box
    [[x1,y1],
     [x2,y2],
     [x3,y3],
     [x4,y4]]
    """
    b1_max_s = b1[np.argmax(b1[:, 0])]
    b1_max_x = b1[np.argmax(b1[:, 1])]
    b1_min_s = b1[np.argmin(b1[:, 0])]
    b1_min_x = b1[np.argmin(b1[:, 1])]

    b2_min_s = b2[np.argmin(b2[:, 0])]
    b2_min_x = b2[np.argmin(b2[:, 1])]
    b2_max_s = b2[np.argmax(b2[:, 0])]
    b2_max_x = b2[np.argmax(b2[:, 1])]
    if b2_min_s[0] <= b1_max_s[0] <= b2_max_s[0] or b2_min_s[0] <= b1_min_s[0] <= b2_max_s[0]:
        if b1_
    else:
        print("b1_max_s and b1_min_s are outside b2_max_s and b2_min_s")
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr

import xcoll.geometry.segments as seg

import xcoll.geometry.c_init as box
# %matplotlib ipympl
#%matplotlib tk

In [ ]:
#L = seg.HalfOpenLineSegment(s1=1,x1=0,cos_t1=np.sqrt(2)/2)
#L.box.l

#L1 = seg.LineSegment(s1=0,x1=0,s2=1,x2=1)
#L1.box.l

#L3 = seg.CircularSegment(R=1,sR=0,xR=0,theta1=np.pi/6,theta2=np.pi/4)

L4 = seg.BezierSegment(s1=0,x1=0,cs1=0.5,cx1=0.5,cs2=0.5,cx2=0.5,s2=1,x2=1) 

In [ ]:
theta_ch = np.arctan2( 0.837257, -0.546810)
theta_b = np.arctan2( 0.546810, 0.837257)
print(np.rad2deg(theta_ch))
print(np.rad2deg(theta_b))

In [ ]:
#L = tr.DriftTrajectory(s0=3, x0=3, xp=np.deg2rad(35))
L_circ = tr.CircularTrajectory(R=1, sR=0, xR=0, s0=0, x0=1, l1=0,l2=np.pi)


L_circ.plot(l1=0, l2=np.pi)



In [ ]:
ran_1 = np.random.normal()
ran_2 = np.random.normal()

L_mcs = tr.MultipleCoulombTrajectory(s0=0.1, x0=1, xp=np.tan(np.deg2rad(30.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
                                     q=1, X0=0.0001, ran_1=ran_1, ran_2=ran_2, l1=0,l2=1)


L_mcs.plot()


In [ ]:
S_circ = seg.CircularSegment(R=1,sR=0.4,xR=1.4,theta1=np.pi/6,theta2=3*np.pi/2, t1=0.,t2=1)
S_circ.plot(t1=0, t2=1)

In [ ]:
overlap = L_mcs.box.overlaps(b2=S_circ.box)
print(f"Overlap: {overlap}")


### Root tests

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
from xcoll.geometry.crossings.find_root import find_root
#%matplotlib ipympl

%matplotlib tk

In [18]:
line = seg.LineSegment(s1=11, x1=13, s2=13, x2=12)
halfopen = seg.HalfOpenLineSegment(s1=10, x1=11, cos_t1=np.tan(np.deg2rad(30.)))
drift = tr.DriftTrajectory(s0=10., x0=11, xp=np.deg2rad(60.0))
mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
                                     q=1, X0=0.0001, ran_1=np.random.normal(), ran_2=np.random.normal(), l1=0,l2=1)

locseg = seg.segment.LocalSegment(line)
loctraj = tr.trajectory.LocalTrajectory(mcs)

#traj = tr.trajectory.LocalTrajectory.from_dict(drift.to_dict())

In [ ]:
# This is failing due to the init of half open. idk why. 
locseg = seg.segment.LocalSegment(halfopen)

In [ ]:
# just for plotting
plt.plot([line.box.s1, line.box.s2, line.box.s3, line.box.s4, line.box.s1],
         [line.box.x1, line.box.x2, line.box.x3, line.box.x4, line.box.x1], 'b-' )
plt.plot([drift.box.s1, drift.box.s2, drift.box.s3, drift.box.s4, drift.box.s1],
         [drift.box.x1, drift.box.x2, drift.box.x3, drift.box.x4, drift.box.x1], 'r-' )
plt.plot([mcs.box.s1, mcs.box.s2, mcs.box.s3, mcs.box.s4, mcs.box.s1],
         [mcs.box.x1, mcs.box.x2, mcs.box.x3, mcs.box.x4, mcs.box.x1], 'g-' )
plt.plot([halfopen.box.s1, halfopen.box.s2, halfopen.box.s3, halfopen.box.s4, halfopen.box.s1],
         [halfopen.box.x1, halfopen.box.x2, halfopen.box.x3, halfopen.box.x4, halfopen.box.x1], 'm-' )
plt.axis('equal')
plt.grid()
plt.show()

In [ ]:
# This calls the exposed find_crossing method in find_root.h 
# I have commented out all the analytical stuff, so it will go straight to the numerical thing. 
# In C, in the final version, I want to remove the dependency of the python class
# for finding the l and t of the root, and use pointers or an array or something to store/return them
# to the "main" function where the scattering is actually taking place.
check = find_root()
check.find_crossing(seg=locseg, traj=loctraj)

In [ ]:
# This is just prev. testing for checking that Newton works.
import xobjects as xo
guess_l = xo.Float64(0.01)
guess_t = xo.Float64(0.3)
check = find_root()

#guess_t, guess_l = check.find_guess_l_and_t(traj=drift, seg=line)
check.newton(seg=locseg, traj=loctraj, guess_l=guess_l, guess_t=guess_t)

In [ ]:
# The solutions are saved in the python class atm for testing. 
print(check.solution_l, check.solution_t)

# Trajectory Tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
#import xcoll.geometry.c_init as box
#%matplotlib ipympl

%matplotlib tk

In [ ]:

kwargs = {'s0': [-3, 3, 0.1], 'x0': [-3, 3, 0.1], 'theta0': [np.deg2rad(-89), np.deg2rad(89), np.deg2rad(10)]}

tr.DriftTrajectory._inspect(**kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.]
          , 'xR': [-3, 3, 0.], 's0': [-2, 2, 1.5], 'x0': [-2, 2, -1.5]}
tr.CircularTrajectory._inspect(**kwargs)

In [ ]:
ran_1 = np.random.normal()
ran_2 = np.random.normal()

#fig, _ = tr.MultipleCoulombTrajectory(s0=0.1, x0=1, xp=np.tan(np.deg2rad(30.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
#                                      q=1, X0=0.0001, ran_1=ran_1, ran_2=ran_2).plot()


kwargs = {'s0': [-3, 3, 0.1], 'x0': [-3, 3, 0.1], 'xp': [np.deg2rad(-180), np.deg2rad(180), np.deg2rad(10)], 'pc':[1e9, 1e9, 1e9],
          'beta': [1.e9/np.sqrt(0.938e9**2 + 1.e9**2), 1.e9/np.sqrt(0.938e9**2 + 1.e9**2), 1.e9/np.sqrt(0.938e9**2 + 1.e9**2)],
          'q': [1, 1, 1], 'X0': [0.0001, 0.001, 0.0001], 'ran_1': [ran_1, ran_1, ran_1], 'ran_2': [ran_2, ran_2, ran_2]}


tr.MultipleCoulombTrajectory._inspect(**kwargs)

In [ ]:
fig, _ =  tr.CircularTrajectory(s0=0, x0=-1, sR=0, xR=0).plot(l1=-0.5, l2=np.pi/2-0.01)


# Segment Tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
import xcoll.geometry.c_init as box
# %matplotlib ipympl
%matplotlib tk

In [ ]:
kwargs = {'s1': [-3, 3, 1.968750e-01], 'x1': [-3, 3, 1.687500e-01], 's2': [-3, 3, -6.468750e-01], 'x2': [-3, 3, -6.468750e-01]}


seg.LineSegment._inspect(**kwargs)



In [ ]:
# make a line segment box to control the angle
s1 =  1.968750e-01
x1 = 1.687500e-01
s2 = -6.468750e-01
x2 = -6.468750e-01
sin_b = (x2 - x1) / np.sqrt((x2 - x1)*(x2 - x1) + (s2 - s1)*(s2 - s1))
cos_b = (s2 - s1) / np.sqrt((x2 - x1)*(x2 - x1) + (s2 - s1)*(s2 - s1))
l = np.sqrt((s2 - s1)*(s2 - s1) + (x2 - x1)*(x2 - x1))   
w = 0.                                             
rC = np.sqrt(s1*s1 + x1*x1)
sin_tC = x1 / rC                       
cos_tC = s1 / rC
projL = rC*(cos_b*cos_tC + sin_b*sin_tC)
projW = rC*(cos_b*sin_tC - sin_b*cos_tC) 
b_line = box.BoundingBox(rC=rC, sin_tC=sin_tC, cos_tC=cos_tC, sin_tb=sin_b, cos_tb=cos_b, l=l,w=w, proj_l=projL, proj_w=projW)

In [ ]:
print(f"rC = {b_line.rC}, sin_tC = {b_line.sin_tC}, \n cos_tC = {b_line.cos_tC}, l = {b_line.l}, w = {b_line.w}, \nsin_tb = {b_line.sin_tb}, cos_tb = {b_line.cos_tb}")
print(f"box s1: {b_line.s1}, box x1: {b_line.x1}, box s2: {b_line.s2}, box x2: {b_line.x2}")
print(f"calc: { b_line.rC * b_line.cos_tC + b_line.l * (b_line.cos_tb)}")

In [ ]:
# line unit vector
dx = sin_tC
dy = cos_tC

# half-length vector
hx = dx * (l / 2)
hy = dy * (l / 2)

# center coordinates
cx = rC  # or s-axis? depends on your frame
cy = 0   # adjust if your center is not on x=0

# endpoints along the line
x1_line = cx - hx
y1_line = cy - hy
x2_line = cx + hx
y2_line = cy + hy

# perpendicular vector
px = sin_b
py = cos_b
w2 = w / 2

# compute four corners
corners = [
    (x1_line - w2*px, y1_line - w2*py),
    (x2_line - w2*px, y2_line - w2*py),
    (x2_line + w2*px, y2_line + w2*py),
    (x1_line + w2*px, y1_line + w2*py),
]

# plot
import matplotlib.pyplot as plt
xs, ys = zip(*corners, corners[0])  # close box
plt.plot(xs, ys, 'b-')
plt.plot([x1_line, x2_line], [y1_line, y2_line], 'r-', lw=2)  # segment line
plt.plot([s2, s1], [x2, x1], 'r-')
plt.axis('equal')
plt.show()


In [ ]:
kwargs = {'s1': [-3, 3, 0], 'x1': [-3, 3, 0], 'theta1': [-2*np.pi, 2*np.pi, np.pi/3]}
seg.HalfOpenLineSegment._inspect(**kwargs)

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3],
          'theta1': [-2*np.pi, 3*np.pi, -np.pi], 'theta2': [-2*np.pi, 3*np.pi, 0.],}
seg.CircularSegment._inspect(**kwargs)

# 1. Providing the centre, radius, and angles: CircularSegment(R=..., sR=..., xR=..., theta1=..., theta2=...)
# 2. Providing the start point, radius, and angles: CircularSegment(R=..., s1=..., x1=..., theta1=..., theta2=...)
# 3. Providing the end point, radius, and angles: CircularSegment(R=..., s2=..., x2=..., theta1=..., theta2=...)
# 4. Providing the start and end point, and the (possibly negative) curvature: CircularSegment(k=..., s1=..., x1=..., s2=..., x2=...)
# 5. Providing the centre, start point, and an angular shift: CircularSegment(sR=..., xR=..., s1=..., x1=..., delta_theta=...)
# 6. Providing the centre, end point, and an angular shift: CircularSegment(sR=..., xR=..., s2=..., x2=..., delta_theta=...)

In [ ]:
ci_negpi_0 = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=-np.pi, theta2=0., t1=0, t2=1)
ci_negpi_0.plot()

#box is  wrong

In [ ]:
ci_negpi_pi = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=-np.pi, theta2=np.pi, t1=0, t2=1)
ci_negpi_pi.plot()

#box is wrong

In [ ]:
ci_negpi_30 = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=-np.pi, theta2=np.pi/6, t1=0, t2=1)
ci_negpi_30.plot()



In [ ]:
ci_0_pi = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=0., theta2=np.pi, t1=0, t2=1)
ci_0_pi.plot()

# box is very wrong

In [ ]:
ci_0_30 = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=0., theta2=np.pi/6, t1=0, t2=1)
ci_0_30.plot()


In [ ]:

ci_120_pi = seg.CircularSegment(R=1.2, s1=0.2, x1 = -0.3, theta1=2.0, theta2=np.pi, t1=0, t2=1)
ci_120_pi.plot()

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 's1': [-3, 3, 0.2], 'x1': [-3, 3, -0.3],
          'theta1': [-np.pi, np.pi, -np.pi/3], 'theta2': [-np.pi, np.pi, np.pi/3]}


seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 's2': [-3, 3, 0.2], 'x2': [-3, 3, -0.3],
          'theta1': [-np.pi, np.pi, -np.pi/3], 'theta2': [-np.pi, 2*np.pi, np.pi/3],}

seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'k': [-1, 1, 0.1], 's1': [-3, 3, 0.2], 'x1': [-3, 3, -0.3], 's2': [-3, 3, 0.7], 'x2': [-3, 3, 0.5]}

# seg.CircularSegment._inspect(**kwargs)
seg.CircularSegment._inspect(plot_bounding_box=False, plot_control_points=False, **kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3], 's1': [-3, 3, 0.7], 'x1': [-3, 3, 0.5],
          'delta_theta': [-2*np.pi, 2*np.pi, np.pi/3]}

seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3], 's1': [-3, 3, 0.7], 'x1': [-3, 3, 0.5],
          'delta_theta': [-2*np.pi, 2*np.pi, -np.pi/3]}

In [ ]:
kwargs = {
    's1': [-3, 3, 0],
    'x1': [-3, 3, 0],
    'cs1': [-3, 3, 0],
    'cx1': [-3, 3, 1],
    'cs2': [-3, 3, 1],
    'cx2': [-3, 3, 0],
    's2': [-3, 3, 1],
    'x2': [-3, 3, 1],
}

seg.BezierSegment._inspect(**kwargs)

# Shape

In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext wurlitzer
%matplotlib ipympl

import numpy as np
import xobjects as xo

import xcoll as xc
from xcoll.geometry.segments import get_max_crossings
from xcoll.geometry.trajectories import DriftTrajectory

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def check_drift(shape, plot_object=False, xlim=(-0.5, 1.5), ylim=(-0.5, 1.5)):
    to_remove = []
    s_val = np.linspace(*xlim, 200)
    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    if plot_object:
        shape.plot(axes=ax)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect('equal')

    n_hit = np.zeros(1, dtype=np.int8)
    s = np.zeros(get_max_crossings(shape, DriftTrajectory), dtype=np.float64)
    # s = np.zeros(8, dtype=np.float64)

    def update_plot(s0, x0, part_theta):
        # Clean the previous trajectory and points
        while to_remove:
            last_point = to_remove.pop()  # Get the last added point
            last_point.remove()
        m = np.tan(np.radians(part_theta))
        # Draw drift trajectory and point
        spread = 10 # degrees
        line, = ax.plot(s_val, m*(s_val - s0) + x0, c='k')
        to_remove.append(line)
        scatter = ax.scatter(s0, x0, c='r', s=20)
        to_remove.append(scatter)
        m1 = np.tan(np.radians(part_theta+spread))
        m2 = np.tan(np.radians(part_theta-spread))
        line = ax.fill_between(s_val, m1*(s_val - s0) + x0, m2*(s_val - s0) + x0, alpha=0.2)
        to_remove.append(line)
        # Create hits for different slopes (small cone around the trajectory)
        hits_s = []
        hits_x = []
        for sp in np.linspace(-spread, spread, 100):
            n_hit[0] = 0
            m = np.tan(np.radians(part_theta + sp))
            if isinstance(shape, xc.Shape2DV):
                shape.crossing_drift(n_hit=n_hit, s=s, s0=s0, x0=x0, xm=m, y0=0, ym=0)
            else:
                shape.crossing_drift(n_hit=n_hit, s=s, s0=s0, x0=x0, xm=m)
            hits_s += list(s[:n_hit[0]])
            hits_x += list(x0 + m*(s[:n_hit[0]] - s0))
        # for i in range(n_hit[0]):
        scatter = ax.scatter(hits_s, hits_x, c='g', s=5)
        # to_remove.append(scatter)
        fig.canvas.draw()  # Redraw the canvas to show the new point


    def onclick(event):
        s0, x0 = event.xdata, event.ydata
        if s0 is not None and x0 is not None:  # Ensure the click is inside the plot
            update_plot(s0, x0, part_theta_slider.value)


    fig.canvas.mpl_connect('button_press_event', onclick)
    part_theta_slider = widgets.IntSlider(min=-89, max=89, step=1, value=0)

    @interact(part_theta=part_theta_slider)
    def check_linesegments(part_theta):
        if to_remove:
            scatter = to_remove[1]  # The second point is the clicked scatter point
            s0, x0 = scatter.get_offsets()[0, 0], scatter.get_offsets()[0, 1]

            # Update the plot with the new slope based on the slider
            update_plot(s0, x0, part_theta)

In [ ]:
shape1 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
                     xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)])
shape2 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                     xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                     xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                     xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                     xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                     xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                     xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)])
shape3 = xc.Shape2DV([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                      xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                      xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                      xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                      xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                      xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                      xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4),
                      xc.HalfOpenLineSegment(s=1.8, x=-0.5, t=5*np.pi/8),
                      xc.CircularSegment(R=0.3, s=1.8-0.3*np.cos(np.pi/8), x=-0.5-0.3*np.sin(np.pi/8), t1=-7*np.pi/8, t2=np.pi/8),
                      xc.HalfOpenLineSegment(s=1.8-0.6*np.cos(np.pi/8), x=-0.5-0.6*np.sin(np.pi/8), t=5*np.pi/8)],
                     vlimit=[-0.1, 0.1])
shape4 = xc.Shape2DV([xc.CircularSegment(R=1, s=0, x=0, t1=0, t2=2*np.pi/3),
                      xc.CircularSegment(R=1, s=0, x=0, t1=2*np.pi/3, t2=4*np.pi/3),
                      xc.CircularSegment(R=1, s=0, x=0, t1=4*np.pi/3, t2=2*np.pi)],
                     vlimit=[-0.1, 0.1])
R1 = 0.5
R2 = 2
t = np.arcsin(R1/R2)
shape5 = xc.Shape2D([xc.CircularSegment(R=R1, s=0, x=0, t1=np.pi/2, t2=-np.pi/2),
                     xc.CircularSegment(R=R2, s=-R2*np.cos(t), x=0, t1=-t, t2=t)])
shape6 = xc.Shape2D([xc.HalfOpenLineSegment(s=0, x=0, t=np.pi/4),
                     xc.HalfOpenLineSegment(s=0, x=0, t=-np.pi/4)])

In [ ]:
#This is meant to fail

xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
            xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)]).get_vertices()

In [ ]:
fig, _ = shape1.plot3d(); fig.show()
fig, _ = shape2.plot3d(); fig.show()
fig, _ = shape3.plot3d(); fig.show()
fig, _ = shape4.plot3d(); fig.show()
fig, _ = shape5.plot3d(); fig.show()
fig, _ = shape6.plot3d(); fig.show()

In [ ]:
shape_1 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
                     xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)])

check_drift(shape_1, plot_object=False)

In [ ]:
shape_2 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                     xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                     xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                     xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                     xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                     xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                     xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)])
check_drift(shape_2, xlim=(-0.5, 4), ylim=(-0.5,2), plot_object=True)

In [ ]:
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=-np.pi/2, t2=np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=np.pi/2,  t2=-np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=3*np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=2*np.pi)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=-np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=-np.pi/2, t2=0)
seg = xc.CircularSegment(R=1, s=0., x=0., t1=3*np.pi/2,t2=0)

check_drift(xc.Shape2D([seg]), plot_object=True, xlim=(-1.5, 1.5), ylim=(-1.5, 1.5))

In [ ]:
shape_3 = xc.Shape2DV([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                      xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                      xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                      xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                      xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                      xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                      xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)],
                    vlimit=[-0.1, 0.1])
check_drift(shape_3, xlim=(-0.5, 4), ylim=(-0.5,2), plot_object=True)